In [1]:
import pandas as pd
from PIL import Image

In [2]:
data_map = pd.read_csv(r'C:\Users\windows10\Documents\GitHub\X-Ray_Pneumonia\data_map.csv') 
data_map.head()

,image_path,class,split
0,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test
1,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test
2,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test
3,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test
4,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_map['class_encoded'] = le.fit_transform(data_map['class'])

In [4]:
data_map

,image_path,class,split,class_encoded
0,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test,0
1,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test,0
2,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test,0
3,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test,0
4,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,NORMAL,test,0
...,...,...,...,...
5851,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,PNEUMONIA,val,1
5852,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,PNEUMONIA,val,1
5853,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,PNEUMONIA,val,1
5854,C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...,PNEUMONIA,val,1


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import transforms, models

In [6]:
class XRayDataset(Dataset):
    def __init__(self, df_split, transform=None):
        self.df_split = df_split
        self.transform = transform
    def __len__(self):
        return len(self.df_split)
    def __getitem__(self, index):
        img_path = self.df_split.iloc[index]['image_path']
        image = Image.open(img_path).convert('L')
        label = self.df_split.iloc[index]['class_encoded']
        if self.transform:
            image_trans = self.transform(image)
        return image_trans, label


In [14]:
train_dataset = XRayDataset(data_map[data_map["split"] == "train"], transform=transforms.ToTensor())
val_dataset = XRayDataset(data_map[data_map["split"] == "val"], transform=transforms.ToTensor())

In [15]:
data_map.iloc[827]

image_path       C:\Users\windows10\Documents\GitHub\X-Ray_Pneu...
class                                                       NORMAL
split                                                        train
class_encoded                                                    0
Name: 827, dtype: object

In [16]:
train_dataset[827]

(tensor([[[0.2157, 0.2471, 0.2784,  ..., 0.0980, 0.0471, 0.0196],
          [0.2039, 0.2392, 0.2784,  ..., 0.0863, 0.0353, 0.0078],
          [0.1922, 0.2275, 0.2745,  ..., 0.0667, 0.0235, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0039],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0039],
          [0.0039, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0039]]]),
 np.int64(0))

In [17]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  

In [19]:
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\windows10/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:38<00:00, 1.20MB/s]


In [ ]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)

In [24]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [26]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [28]:
from tqdm import tqdm

In [ ]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, total=len(train_loader)):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        pred = model(images)
        loss_val = loss(pred, labels)
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

100%|██████████| 163/163 [06:34<00:00,  2.42s/it]


Epoch 1, Loss: 0.1311


 94%|█████████▍| 154/163 [05:39<00:19,  2.21s/it]

In [ ]:
torch.save(model.state_dict(), 'C:\Users\windows10\Documents\GitHub\X-Ray_Pneumonia\models\xray_resnet18.pth')

In [ ]:
img_path = r'C:\Users\windows10\Documents\GitHub\X-Ray_Pneumonia\Dataset\images\person100_bacteria_2931.jpeg'
image = Image.open(img_path).convert('L')
img_trans = transforms.ToTensor()(image).unsqueeze(0).to(device)
pred = model(img_trans)
pred_class = torch.argmax(pred, dim=1).item()
le.inverse_transform([pred_class])